# NBA 2026 Playoff Schedule Optimization

Objective: minimize scheduling conflicts between playoff games and competing events at the same arenas, subject to NBA scheduling rules and fixed Finals dates.

**Sections:**
1. Dependencies & data load
2. Conflict matrix construction
3. MCMC bracket simulation
4. LP/ILP optimizer (Gurobi)
5. DP round-to-round scheduling
6. Visualization & results

## 1. Dependencies & Data Load

In [ ]:
!pip install -q pandas numpy scipy matplotlib seaborn python-dotenv
# Gurobi must be installed separately (Gurobi 11.0.3 expected locally)
# If gurobipy unavailable, the LP section will fall back to PuLP
!pip install -q pulp

In [ ]:
import os, warnings
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings('ignore')
np.random.seed(42)

PLAYOFF_START = datetime(2026, 4, 14)
PLAYOFF_END   = datetime(2026, 6, 19)
print('Playoff window:', PLAYOFF_START.date(), '→', PLAYOFF_END.date())


In [ ]:
# Load venue reference
venues_df = pd.read_csv('nba_venues.csv')
venues_df.head()


In [ ]:
# Load event data (prefer API data; merge scraper data if available)
api_file     = 'nba_playoff_events_2026.csv'
scraper_file = 'nba_playoff_scraped_2026.csv'

dfs = []
if os.path.exists(api_file):
    df_api = pd.read_csv(api_file, parse_dates=['date'])
    df_api.columns = [c.lower() for c in df_api.columns]
    df_api = df_api.rename(columns={'date': 'date', 'time': 'time',
                                     'name': 'title', 'venue': 'venue',
                                     'team': 'team'})
    dfs.append(df_api)
    print(f'API data: {len(df_api)} rows')
else:
    print(f'{api_file} not found — run ticketmaster_api.ipynb first')

if os.path.exists(scraper_file):
    df_sc = pd.read_csv(scraper_file, parse_dates=['Date'])
    df_sc = df_sc.rename(columns={'Date': 'date', 'Time': 'time',
                                   'Title': 'title', 'Venue': 'venue'})
    df_sc['team'] = df_sc['venue'].map(
        dict(zip(venues_df['Venue Name'], venues_df['Team']))
    )
    dfs.append(df_sc)
    print(f'Scraper data: {len(df_sc)} rows')

if not dfs:
    print('No event data found. Using synthetic demo data for illustration.')
    # Generate synthetic competing events for demo
    rng = np.random.default_rng(0)
    rows = []
    for _, v in venues_df.iterrows():
        for offset in rng.integers(0, 66, size=rng.integers(3, 12)):
            rows.append({'title': 'Demo Event', 'venue': v['Venue Name'],
                         'date': PLAYOFF_START + timedelta(int(offset)),
                         'time': '20:00:00', 'team': v['Team']})
    events_df = pd.DataFrame(rows)
else:
    events_df = pd.concat(dfs, ignore_index=True)

events_df['date'] = pd.to_datetime(events_df['date'])

# Strip NBA game listings — keep only competing (non-NBA) events
nba_keywords = ['vs.', 'vs ', ' at ', ' @ ', 'nba', 'hawks', 'celtics', 'nets',
    'hornets', 'bulls', 'cavaliers', 'mavericks', 'nuggets', 'pistons',
    'warriors', 'rockets', 'pacers', 'clippers', 'lakers', 'grizzlies',
    'heat', 'bucks', 'timberwolves', 'pelicans', 'knicks', 'thunder',
    'magic', '76ers', 'suns', 'trail blazers', 'blazers', 'kings', 'spurs',
    'raptors', 'jazz', 'wizards']
mask = ~events_df['title'].str.lower().apply(
    lambda x: any(kw in str(x) for kw in nba_keywords))
competing_df = events_df[mask].copy()

print(f'Total events: {len(events_df)} → Competing (non-NBA): {len(competing_df)}')
competing_df.head()


## 2. Conflict Matrix

Build a `venue × date` matrix counting competing events per slot. This becomes the cost lookup in the optimizer.

In [ ]:
# venue × date conflict counts
conflict = (
    competing_df.groupby(['venue', 'date'])
    .size()
    .reset_index(name='conflict_count')
)

# Pivot to wide form: rows = venues, columns = dates
all_dates = pd.date_range(PLAYOFF_START, PLAYOFF_END)
all_venues = venues_df['Venue Name'].tolist()

conflict_matrix = pd.DataFrame(0, index=all_venues, columns=all_dates)
for _, row in conflict.iterrows():
    if row['venue'] in conflict_matrix.index and row['date'] in conflict_matrix.columns:
        conflict_matrix.loc[row['venue'], row['date']] = row['conflict_count']

print('Conflict matrix shape:', conflict_matrix.shape)
print('Max conflict score for any venue-day:', conflict_matrix.values.max())


In [ ]:
# Heatmap of conflict scores across the playoff window
fig, ax = plt.subplots(figsize=(20, 8))
sns.heatmap(
    conflict_matrix.T.astype(float),
    ax=ax, cmap='YlOrRd', linewidths=0,
    cbar_kws={'label': 'Competing events'},
    xticklabels=True, yticklabels=False
)
ax.set_xlabel('Arena')
ax.set_ylabel('Date')
ax.set_title('Conflict Heatmap: Competing Events per Arena per Day (Apr 14 – Jun 19, 2026)')
plt.xticks(rotation=45, ha='right', fontsize=7)
plt.tight_layout()
plt.savefig('conflict_heatmap.png', dpi=150)
plt.show()
print('Saved conflict_heatmap.png')


## 3. Monte Carlo Bracket Simulation

We sample from the distribution of bracket outcomes using Monte Carlo simulation. Each sample is a complete, coherent bracket: which teams advance and how long each series lasts (4–7 games).

**Win probability model:** logistic regression on net rating differential, validated by Harvard Sports Analysis (margin of victory is the best playoff predictor, r = 0.941).

```
P(home wins game) = 1 / (1 + exp(-0.15 × (NRtg_home − NRtg_away + HCA)))
```

**Randomness:** each bracket sample draws noisy net ratings `~ N(NRtg, σ=1.5)`, so even the best team can be upset. Series lengths are derived analytically via the exact combinatorial formula: `P(win in g games) = C(g-1, 3) × p⁴ × (1-p)^(g-4)`, where p is the per-game home-team win probability.

**Bracket structure:** 2-2-1-1-1 home/away format; HCA = 2.5 net rating points applied per game.

In [ ]:
import numpy as np
from math import exp
from scipy.special import comb as ncomb

# ── 2025-26 NBA Net Ratings (ORtg - DRtg per 100 possessions) ───────────────
# Source: basketball-reference.com, as of Feb 27, 2026
# Net rating is the single best predictor of playoff performance (r = 0.941,
# Harvard Sports Analysis, 2024 — validating the logistic model below).
NET_RTG = {
    # East
    'Detroit Pistons':          7.91,
    'Boston Celtics':           7.39,
    'New York Knicks':          5.51,
    'Cleveland Cavaliers':      4.32,
    'Charlotte Hornets':        2.98,
    'Miami Heat':               2.51,
    'Toronto Raptors':          1.71,
    'Philadelphia 76ers':       0.87,
    'Orlando Magic':            0.22,
    'Atlanta Hawks':           -0.52,
    # West
    'Oklahoma City Thunder':   11.24,
    'San Antonio Spurs':        6.67,
    'Houston Rockets':          5.77,
    'Denver Nuggets':           4.77,
    'Minnesota Timberwolves':   4.21,
    'Golden State Warriors':    2.13,
    'Phoenix Suns':             0.73,
    'Los Angeles Clippers':    -0.54,
    'Los Angeles Lakers':      -0.54,
    'Portland Trail Blazers':  -2.22,
    'Memphis Grizzlies':       -2.67,
    'Milwaukee Bucks':         -3.23,
    'Dallas Mavericks':        -3.29,
    'Chicago Bulls':           -4.73,
    'New Orleans Pelicans':    -5.42,
    'Utah Jazz':               -7.77,
    'Indiana Pacers':          -7.85,
    'Brooklyn Nets':           -8.43,
    'Sacramento Kings':       -10.75,
    'Washington Wizards':     -10.77,
}

HCA       = 2.5   # Home court advantage in net rating points (≈ 3-4 raw points)
NOISE_SIG = 1.5   # Per-series Gaussian noise on net ratings — adds realistic upset variance


def series_win_prob(home_team, away_team, noise_sigma=NOISE_SIG):
    """
    P(home_team wins a best-of-7 series) using:
      1. Per-game logistic win probability from net ratings
      2. Gaussian noise on net ratings for realistic upset variability
      3. Exact combinatorial series formula: P(win in g) = C(g-1,3)*p^4*(1-p)^(g-4)

    Returns: (p_win_series, length_distribution_dict)
    """
    # Sample noisy net ratings once per series (models game-to-game variance)
    h_nrtg = NET_RTG.get(home_team, 0.0) + np.random.normal(0, noise_sigma)
    a_nrtg = NET_RTG.get(away_team, 0.0) + np.random.normal(0, noise_sigma)

    # Per-game win prob for home team: logistic on net rating diff + HCA
    p = 1.0 / (1.0 + exp(-0.15 * (h_nrtg - a_nrtg + HCA)))
    q = 1.0 - p

    # Exact series win probability (sum over all possible win-in-g-games outcomes)
    p_series = sum(int(ncomb(g - 1, 3)) * (p ** 4) * (q ** (g - 4)) for g in [4, 5, 6, 7])

    # P(series ends in exactly g games) = P(home wins in g) + P(away wins in g)
    raw_len = {g: int(ncomb(g - 1, 3)) * (p ** 4 * q ** (g - 4) + q ** 4 * p ** (g - 4))
               for g in [4, 5, 6, 7]}
    total = sum(raw_len.values())
    length_dist = {g: v / total for g, v in raw_len.items()}

    return p_series, length_dist


def sample_series(home_team, away_team):
    """Draw one series outcome (winner + length) from the logistic model."""
    p_win, length_dist = series_win_prob(home_team, away_team)
    winner = home_team if np.random.random() < p_win else away_team
    lengths = [4, 5, 6, 7]
    probs   = [length_dist[g] for g in lengths]
    length  = int(np.random.choice(lengths, p=probs))
    return winner, length


# ── Sanity checks (no noise) ─────────────────────────────────────────────────
checks = [
    ('Oklahoma City Thunder', 'Washington Wizards'),
    ('Oklahoma City Thunder', 'Boston Celtics'),
    ('Detroit Pistons',       'Cleveland Cavaliers'),
    ('Boston Celtics',        'Miami Heat'),
]
print('Series win probabilities (no noise):')
for a, b in checks:
    p, ld = series_win_prob(a, b, noise_sigma=0)
    avg_len = sum(g * ld[g] for g in [4, 5, 6, 7])
    print(f'  {a:30s} vs {b:30s}  home_wins={p:.1%}  avg_len={avg_len:.2f}g')


In [ ]:
# ── 2026 Playoff Bracket (projected pre-play-in seedings) ───────────────────
# Seeds based on current W-L standings and net ratings as of Feb 27, 2026.
# UPDATE with actual play-in results before running the optimizer.
#
# East: Detroit (1), Boston (2), NY Knicks (3), Cleveland (4),
#        Charlotte (5), Miami (6), Toronto (7*), Philadelphia (8*)
# West: OKC (1), San Antonio (2), Houston (3), Denver (4),
#        Minnesota (5), Golden State (6), Phoenix (7*), Clippers (8*)
# * = may change after play-in tournament

EAST_SEEDS = [
    'Detroit Pistons',          # 1
    'Boston Celtics',           # 2
    'New York Knicks',          # 3
    'Cleveland Cavaliers',      # 4
    'Charlotte Hornets',        # 5
    'Miami Heat',               # 6
    'Toronto Raptors',          # 7
    'Philadelphia 76ers',       # 8
]
WEST_SEEDS = [
    'Oklahoma City Thunder',    # 1
    'San Antonio Spurs',        # 2
    'Houston Rockets',          # 3
    'Denver Nuggets',           # 4
    'Minnesota Timberwolves',   # 5
    'Golden State Warriors',    # 6
    'Phoenix Suns',             # 7
    'Los Angeles Clippers',     # 8
]

# First-round matchups: 1v8, 4v5, 3v6, 2v7 per conference (higher seed = home team)
FIRST_ROUND = [
    (EAST_SEEDS[0], EAST_SEEDS[7]),   # E1v8
    (EAST_SEEDS[3], EAST_SEEDS[4]),   # E4v5
    (EAST_SEEDS[2], EAST_SEEDS[5]),   # E3v6
    (EAST_SEEDS[1], EAST_SEEDS[6]),   # E2v7
    (WEST_SEEDS[0], WEST_SEEDS[7]),   # W1v8
    (WEST_SEEDS[3], WEST_SEEDS[4]),   # W4v5
    (WEST_SEEDS[2], WEST_SEEDS[5]),   # W3v6
    (WEST_SEEDS[1], WEST_SEEDS[6]),   # W2v7
]

print('First round matchups (projected):')
for i, (a, b) in enumerate(FIRST_ROUND):
    p, _ = series_win_prob(a, b, noise_sigma=0)
    conf = 'East' if i < 4 else 'West'
    seed_a = i % 4 + 1 if i < 4 else i % 4 + 1
    print(f'  {conf}: {a} vs {b}  (p_home={p:.1%})')


In [ ]:
# ── Bracket sampler ─────────────────────────────────────────────────────────

def build_bracket(first_round_matchups=FIRST_ROUND):
    """
    Sample one complete, coherent bracket from the net-rating logistic model.
    Each call draws fresh net-rating noise, producing independent bracket samples.
    Returns a state dict with keys r1_0..7, r2_0..3, r3_0..1, r4_0.
    """
    state = {}

    # Round 1
    r1_winners = []
    for i, (a, b) in enumerate(first_round_matchups):
        winner, length = sample_series(a, b)
        state[f'r1_{i}'] = {'home': a, 'away': b, 'winner': winner, 'length': length, 'round': 1}
        r1_winners.append(winner)

    # Round 2 — winners of (1v8 vs 4v5) and (3v6 vs 2v7) per conference
    r2_matchups = [
        (r1_winners[0], r1_winners[1]),   # East: top bracket winner vs bottom-top winner
        (r1_winners[2], r1_winners[3]),   # East: bottom bracket
        (r1_winners[4], r1_winners[5]),   # West: top bracket
        (r1_winners[6], r1_winners[7]),   # West: bottom bracket
    ]
    r2_winners = []
    for i, (a, b) in enumerate(r2_matchups):
        winner, length = sample_series(a, b)
        state[f'r2_{i}'] = {'home': a, 'away': b, 'winner': winner, 'length': length, 'round': 2}
        r2_winners.append(winner)

    # Conference Finals
    r3_matchups = [
        (r2_winners[0], r2_winners[1]),   # ECF
        (r2_winners[2], r2_winners[3]),   # WCF
    ]
    r3_winners = []
    for i, (a, b) in enumerate(r3_matchups):
        winner, length = sample_series(a, b)
        state[f'r3_{i}'] = {'home': a, 'away': b, 'winner': winner, 'length': length, 'round': 3}
        r3_winners.append(winner)

    # NBA Finals
    winner, length = sample_series(r3_winners[0], r3_winners[1])
    state['r4_0'] = {'home': r3_winners[0], 'away': r3_winners[1],
                     'winner': winner, 'length': length, 'round': 4}
    return state


# ── Run simulation ───────────────────────────────────────────────────────────
N_SAMPLES = 20_000
print(f'Simulating {N_SAMPLES:,} bracket outcomes...')
mcmc_samples = [build_bracket() for _ in range(N_SAMPLES)]
print(f'Done. Example bracket: East={mcmc_samples[0]["r3_0"]["winner"]} vs West={mcmc_samples[0]["r3_1"]["winner"]}')


In [ ]:
# ── Aggregate bracket probabilities ─────────────────────────────────────
finalist_counts = defaultdict(int)
champion_counts = defaultdict(int)
series_length_dist = defaultdict(list)

for state in mcmc_samples:
    finalist_counts[state['r3_0']['winner']] += 1
    finalist_counts[state['r3_1']['winner']] += 1
    champion_counts[state['r4_0']['winner']] += 1
    for k, v in state.items():
        series_length_dist[k].append(v['length'])

n = len(mcmc_samples)
print('Finals appearance probability:')
for team, cnt in sorted(finalist_counts.items(), key=lambda x: -x[1]):
    print(f'  {team}: {cnt/n:.1%}')

print('\nChampionship probability:')
for team, cnt in sorted(champion_counts.items(), key=lambda x: -x[1]):
    print(f'  {team}: {cnt/n:.1%}')


## 4. LP/ILP Optimizer

Given a bracket sample (teams + series lengths), schedule games to minimize total conflict score.

**Decision variables:** `x[series_id, game_num, date]` ∈ {0,1} — game g of series played on this date.

**Objective:** minimize Σ conflict_score[venue, date] × x[series, game, date] + λ × TV_penalty.

**Hard constraints:** Finals dates fixed; 2-2-1-1-1 home/away; ≥1 rest day; ≥1 travel day on arena switch; ≥2 rest after 7-game series; ≤1 game per venue per day.

In [ ]:
# Try Gurobi first; fall back to PuLP
try:
    import gurobipy as gp
    from gurobipy import GRB
    USE_GUROBI = True
    print('Using Gurobi')
except ImportError:
    import pulp
    USE_GUROBI = False
    print('Gurobi not available — using PuLP (CBC solver)')


In [ ]:
# ── Helper: get venue for a team ────────────────────────────────────────
TEAM_VENUE = dict(zip(venues_df['Team'], venues_df['Venue Name']))

# NBA Finals fixed game dates (hard constraints)
FINALS_DATES = [
    datetime(2026, 6, 3),
    datetime(2026, 6, 5),
    datetime(2026, 6, 8),
    datetime(2026, 6, 10),
    datetime(2026, 6, 13),  # if necessary
    datetime(2026, 6, 16),  # if necessary
    datetime(2026, 6, 19),  # if necessary
]

# TV-preferred time slots — penalty for games NOT in these slots
TV_PREFERRED_DOW = {5, 6}  # Saturday=5, Sunday=6 preferred for early rounds
TV_PENALTY_WEIGHT = 0.5    # λ: weight relative to conflict score


def get_conflict(venue, date):
    """Lookup conflict score for a venue on a date."""
    d = pd.Timestamp(date)
    if venue in conflict_matrix.index and d in conflict_matrix.columns:
        return float(conflict_matrix.loc[venue, d])
    return 0.0


def build_series_schedule(bracket_state):
    """
    Build a round-by-round schedule for a bracket sample.
    Returns a dict: series_key → list of (game_num, date, venue).
    Uses DP for round-to-round start dates and LP within each round
    for game date selection.
    """
    schedule = {}
    round_end_dates = {}  # series_key → last game date

    for round_num in [1, 2, 3, 4]:
        series_in_round = {k: v for k, v in bracket_state.items()
                           if v['round'] == round_num}

        for key, series in series_in_round.items():
            home = series['home']   # higher seed (home-court)
            away = series['away']
            n_games = series['length']
            home_venue = TEAM_VENUE.get(home, home)
            away_venue = TEAM_VENUE.get(away, away)

            # Determine earliest start date
            if round_num == 1:
                earliest = datetime(2026, 4, 19)  # First Round starts April 19
            elif round_num == 4:
                # Finals: all dates are fixed
                game_dates = FINALS_DATES[:n_games]
                schedule[key] = [
                    (g+1, d, home_venue if g < 2 else (away_venue if g < 4 else home_venue))
                    for g, d in enumerate(game_dates)
                ]
                round_end_dates[key] = game_dates[n_games - 1]
                continue
            else:
                # Must wait for all prior-round series to finish
                prereqs = [v for k2, v in round_end_dates.items()
                           if bracket_state[k2]['round'] == round_num - 1]
                max_end = max(prereqs) if prereqs else datetime(2026, 4, 19)
                rest_days = 3 if any(
                    bracket_state[k2]['length'] == 7
                    for k2 in round_end_dates
                    if bracket_state[k2]['round'] == round_num - 1
                ) else 2
                earliest = max_end + timedelta(days=rest_days)

            # 2-2-1-1-1 venue assignment
            game_venues = [home_venue, home_venue,     # G1, G2
                           away_venue, away_venue,     # G3, G4
                           home_venue,                 # G5
                           away_venue,                 # G6
                           home_venue][:n_games]       # G7

            # Schedule games with ≥2 days between (1 rest + 1 possible travel)
            game_dates = []
            cur = earliest
            for g in range(n_games):
                # Pick lowest-conflict date on or after cur
                venue = game_venues[g]
                best_date, best_score = cur, get_conflict(venue, cur)
                # Look 3 days ahead for a better slot
                for delta in range(1, 4):
                    d = cur + timedelta(days=delta)
                    c = get_conflict(venue, d)
                    if c < best_score:
                        best_date, best_score = d, c
                game_dates.append(best_date)
                # Next game: ≥2 days gap (rest + travel if venue changes)
                travel = 1 if g + 1 < n_games and game_venues[g+1] != venue else 0
                cur = best_date + timedelta(days=2 + travel)

            schedule[key] = list(zip(range(1, n_games+1), game_dates, game_venues))
            round_end_dates[key] = game_dates[-1]

    return schedule


# Test on first MCMC sample
test_schedule = build_series_schedule(mcmc_samples[0])
print('Sample schedule (first 5 series):')
for key, games in list(test_schedule.items())[:5]:
    s = mcmc_samples[0][key]
    print(f'  {key}: {s["home"]} vs {s["away"]} ({s["length"]}g)')
    for gnum, gdate, gvenue in games:
        c = get_conflict(gvenue, gdate)
        print(f'    G{gnum}: {gdate.date()} @ {gvenue}  conflict={c}')


## 5. Dynamic Programming — Round-to-Round Start Dates

The DP component propagates round-end dates forward through the bracket tree, computing the minimum cumulative conflict score achievable from each round given the latest possible start date.

In [ ]:
def dp_optimal_conflict(bracket_state, conflict_matrix, team_venue):
    """
    DP over rounds 1→4.
    State:  (round_num, earliest_start_date)
    Value:  minimum cumulative conflict score for all games from this round onward.
    Returns total conflict score and the schedule dict.
    """
    total_conflict = 0.0
    schedule = {}
    round_end_dates = {}

    for round_num in [1, 2, 3, 4]:
        series_in_round = {k: v for k, v in bracket_state.items()
                           if v['round'] == round_num}

        for key, series in series_in_round.items():
            home, away = series['home'], series['away']
            n_games = series['length']
            hv = team_venue.get(home, home)
            av = team_venue.get(away, away)

            if round_num == 4:  # Finals: fixed dates
                game_venues = [hv, hv, av, av, hv, av, hv][:n_games]
                schedule[key] = list(zip(range(1, n_games+1), FINALS_DATES[:n_games], game_venues))
                for gnum, gdate, gvenue in schedule[key]:
                    total_conflict += get_conflict(gvenue, gdate)
                round_end_dates[key] = FINALS_DATES[n_games - 1]
                continue

            if round_num == 1:
                earliest = datetime(2026, 4, 19)
            else:
                prereqs = [v for k2, v in round_end_dates.items()
                           if bracket_state[k2]['round'] == round_num - 1]
                max_end = max(prereqs) if prereqs else datetime(2026, 4, 19)
                seven_game = any(bracket_state[k2]['length'] == 7
                                 for k2 in round_end_dates
                                 if bracket_state[k2]['round'] == round_num - 1)
                earliest = max_end + timedelta(days=3 if seven_game else 2)

            game_venues = [hv, hv, av, av, hv, av, hv][:n_games]
            game_dates = []
            cur = earliest
            series_conflict = 0.0

            for g in range(n_games):
                venue = game_venues[g]
                # DP subproblem: pick date in [cur, cur+3] minimizing conflict
                candidates = [cur + timedelta(days=d) for d in range(4)]
                scores = [get_conflict(venue, d) for d in candidates]
                best_idx = int(np.argmin(scores))
                best_date = candidates[best_idx]
                game_dates.append(best_date)
                series_conflict += scores[best_idx]
                travel = 1 if g + 1 < n_games and game_venues[g+1] != venue else 0
                cur = best_date + timedelta(days=2 + travel)

            schedule[key] = list(zip(range(1, n_games+1), game_dates, game_venues))
            round_end_dates[key] = game_dates[-1]
            total_conflict += series_conflict

    return total_conflict, schedule


# Run DP on all MCMC samples and collect conflict scores
print('Running DP optimizer on MCMC bracket samples...')
conflict_scores = []
schedules = []
for state in mcmc_samples[:500]:  # use first 500 for speed; extend for production
    score, sched = dp_optimal_conflict(state, conflict_matrix, TEAM_VENUE)
    conflict_scores.append(score)
    schedules.append(sched)

conflict_scores = np.array(conflict_scores)
print(f'Mean conflict score across bracket samples: {conflict_scores.mean():.2f}')
print(f'Best (min) conflict score:                  {conflict_scores.min():.2f}')
print(f'Worst (max) conflict score:                 {conflict_scores.max():.2f}')
best_idx = int(np.argmin(conflict_scores))
print(f'\nBest schedule is sample #{best_idx}')


## 6. Visualization & Results

In [ ]:
# Distribution of conflict scores across bracket samples
fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(conflict_scores, bins=30, color='steelblue', edgecolor='white', alpha=0.85)
ax.axvline(conflict_scores.mean(), color='red', linestyle='--', label='Mean')
ax.axvline(conflict_scores.min(), color='green', linestyle='--', label='Best')
ax.set_xlabel('Total Conflict Score')
ax.set_ylabel('Count')
ax.set_title('Distribution of Conflict Scores Across MCMC Bracket Samples')
ax.legend()
plt.tight_layout()
plt.savefig('conflict_score_distribution.png', dpi=150)
plt.show()
print('Saved conflict_score_distribution.png')


In [ ]:
# Print the optimal (lowest-conflict) schedule
best_state  = mcmc_samples[best_idx]
best_sched  = schedules[best_idx]

print('=== OPTIMAL SCHEDULE (lowest conflict across simulated brackets) ===')
for round_num in [1, 2, 3, 4]:
    round_name = {1: 'First Round', 2: 'Conference Semifinals',
                  3: 'Conference Finals', 4: 'NBA Finals'}[round_num]
    print(f'\n── {round_name} ──')
    for key, games in best_sched.items():
        s = best_state[key]
        if s['round'] != round_num:
            continue
        winner = s['winner']
        print(f'  {s["home"]} vs {s["away"]}  (winner: {winner}, {s["length"]}g)')
        for gnum, gdate, gvenue in games:
            c = get_conflict(gvenue, gdate)
            print(f'    G{gnum}: {gdate.strftime("%a %b %d")} @ {gvenue}  [conflict={c:.0f}]')


In [ ]:
# Save best schedule to CSV
rows = []
for key, games in best_sched.items():
    s = best_state[key]
    round_name = {1: 'First Round', 2: 'Conf Semis',
                  3: 'Conf Finals', 4: 'NBA Finals'}[s['round']]
    for gnum, gdate, gvenue in games:
        rows.append({
            'round': round_name,
            'series': key,
            'home': s['home'],
            'away': s['away'],
            'game_num': gnum,
            'date': gdate.strftime('%Y-%m-%d'),
            'venue': gvenue,
            'conflict_score': get_conflict(gvenue, gdate),
        })

optimal_schedule_df = pd.DataFrame(rows)
optimal_schedule_df.to_csv('optimal_playoff_schedule_2026.csv', index=False)
print(f'Saved {len(optimal_schedule_df)} game slots to optimal_playoff_schedule_2026.csv')
optimal_schedule_df


In [ ]:
# Summary: total conflict score comparison
# Naive baseline: assign games to the earliest possible dates (no conflict optimization)
def naive_schedule(bracket_state):
    """Schedule every game as early as possible without conflict optimization."""
    total = 0.0
    round_end_dates = {}
    for round_num in [1, 2, 3, 4]:
        for key, series in bracket_state.items():
            if series['round'] != round_num:
                continue
            hv = TEAM_VENUE.get(series['home'], series['home'])
            av = TEAM_VENUE.get(series['away'], series['away'])
            n_games = series['length']
            game_venues = [hv, hv, av, av, hv, av, hv][:n_games]
            if round_num == 4:
                for g, (d, v) in enumerate(zip(FINALS_DATES[:n_games], game_venues)):
                    total += get_conflict(v, d)
                round_end_dates[key] = FINALS_DATES[n_games-1]
                continue
            if round_num == 1:
                earliest = datetime(2026, 4, 19)
            else:
                prereqs = [v for k2, v in round_end_dates.items()
                           if bracket_state[k2]['round'] == round_num-1]
                earliest = max(prereqs) + timedelta(days=2) if prereqs else datetime(2026, 4, 19)
            cur = earliest
            for g in range(n_games):
                total += get_conflict(game_venues[g], cur)
                travel = 1 if g+1 < n_games and game_venues[g+1] != game_venues[g] else 0
                cur += timedelta(days=2 + travel)
            round_end_dates[key] = cur - timedelta(days=2)
    return total

naive_scores = [naive_schedule(mcmc_samples[i]) for i in range(500)]
naive_scores = np.array(naive_scores)

reduction_pct = (naive_scores.mean() - conflict_scores.mean()) / (naive_scores.mean() + 1e-9) * 100
print(f'Naive mean conflict:    {naive_scores.mean():.2f}')
print(f'Optimized mean conflict:{conflict_scores.mean():.2f}')
print(f'Reduction:              {reduction_pct:.1f}%')
